# 01. 합성곱 신경망 (CNN)
> 이미지 분석 등에 주로 사용되는 합성곱 신경망(CNN)에 대해 알아봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 8]
- permalink: /cnn
- exec: colab

### 1. 합성곱 신경망(CNN)이란?

이미지와 같은 데이터들은 2D 이상의 차원을 가집니다. 그러나 우리가 지금까지 배운 다층퍼셉트론(MLP)등의 모델은 이미지와 같은 2차원 데이터를 있는 그대로 처리하지 못합니다. 무슨말인지 알아볼까요? <br><br>

![](https://testerstories.com/files/ai_and_ml/ml-mnist-2d-to-1d.png)